In [112]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from lib.mutation_util import get_tx_datetime
from apate.apate_5 import ApateGraphFeatures

%load_ext autoreload
%autoreload

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
df_ori = pd.read_csv('../adyen-dataset.csv')
df_graph_features = df_ori.copy()[
    ["psp_reference", "merchant", "card_number", "email_address", "ip_address", "has_fraudulent_dispute"]]
df_graph_features["tx_datetime"] = df_ori.parallel_apply(get_tx_datetime, axis=1)  # for each row of df
df_graph_features["tx_datetime"] = pd.to_datetime(df_graph_features["tx_datetime"])
# df_graph_features = df_graph_features[(df_graph_features["ip_address"].notna()) & (df_graph_features["email_address"].notna())]
df_graph_features.head()

,psp_reference,merchant,card_number,email_address,ip_address,has_fraudulent_dispute,tx_datetime
0,79622457320,Merchant A,XldVNN9TQgIA2RE9FTC2TA,3XBRYto3QgX7g4pX-BvNyA,i1oY1kTOXMlsFfoyhFJxyA,False,2021-07-25 13:51:00
1,41293916020,Merchant C,zgRoL8pxX5LJI-SEPUTMvg,klwgzdfrnz_6eGsHSi2prQ,HZkxQvtpScNLgi0fsoSkjA,False,2021-12-11 15:20:00
2,31382555255,Merchant E,T3BF1uOOqgcwaVRb_ujPQg,XkMWeLLmXALPcV3ndwzKJw,HGW5EwBnHVgOrD5xYplBdw,False,2021-10-20 01:56:00
3,13944051314,Merchant D,BMf1zFkJ1xvPMlOxC7RWTw,26Lw20NQtPKb5cyQZBuh1w,8AuFEjK8m-0lxGAOJzOPmg,False,2021-08-30 12:50:00
4,62343575411,Merchant E,jfxS2TIe0gR9vpOUdMKrBw,Y7tv2i85iUadkEIAPIufgg,NaN,False,2021-11-14 19:26:00


In [114]:
df_graph_features.shape

(138701, 7)

## Adapt features to apate features

In [115]:
# ['Global_Date', 'Local_Date', 'CardID', 'MerchantID', 'Target']

In [116]:
df_graph_features["Global_Date"] = df_graph_features["tx_datetime"]
df_graph_features["Local_Date"] = df_graph_features["tx_datetime"]
df_graph_features["Target"] = df_graph_features["has_fraudulent_dispute"]
df_graph_features = df_graph_features[['psp_reference','Global_Date', 'Local_Date', 'card_number', 'merchant', 'ip_address', 'email_address', 'Target']]
df_graph_features.head()

,psp_reference,Global_Date,Local_Date,card_number,merchant,ip_address,email_address,Target
0,79622457320,2021-07-25 13:51:00,2021-07-25 13:51:00,XldVNN9TQgIA2RE9FTC2TA,Merchant A,i1oY1kTOXMlsFfoyhFJxyA,3XBRYto3QgX7g4pX-BvNyA,False
1,41293916020,2021-12-11 15:20:00,2021-12-11 15:20:00,zgRoL8pxX5LJI-SEPUTMvg,Merchant C,HZkxQvtpScNLgi0fsoSkjA,klwgzdfrnz_6eGsHSi2prQ,False
2,31382555255,2021-10-20 01:56:00,2021-10-20 01:56:00,T3BF1uOOqgcwaVRb_ujPQg,Merchant E,HGW5EwBnHVgOrD5xYplBdw,XkMWeLLmXALPcV3ndwzKJw,False
3,13944051314,2021-08-30 12:50:00,2021-08-30 12:50:00,BMf1zFkJ1xvPMlOxC7RWTw,Merchant D,8AuFEjK8m-0lxGAOJzOPmg,26Lw20NQtPKb5cyQZBuh1w,False
4,62343575411,2021-11-14 19:26:00,2021-11-14 19:26:00,jfxS2TIe0gR9vpOUdMKrBw,Merchant E,NaN,Y7tv2i85iUadkEIAPIufgg,False


In [117]:
df_graph_features.shape

(138701, 8)

In [118]:
df_graph_features_learning = df_graph_features.loc[(df_graph_features["Global_Date"].dt.month >= 1) & (df_graph_features["Global_Date"].dt.month <= 9)].copy()
df_graph_features_learning = df_graph_features_learning[(df_graph_features_learning["ip_address"].notna()) & (df_graph_features_learning["email_address"].notna())]
# df_graph_features_learning['email_address'] = df_graph_features_learning['email_address'].fillna('unknown_email')
# df_graph_features_learning['ip_address'] = df_graph_features_learning['ip_address'].fillna('unknown_ip')
df_graph_features_learning.head()

,psp_reference,Global_Date,Local_Date,card_number,merchant,ip_address,email_address,Target
0,79622457320,2021-07-25 13:51:00,2021-07-25 13:51:00,XldVNN9TQgIA2RE9FTC2TA,Merchant A,i1oY1kTOXMlsFfoyhFJxyA,3XBRYto3QgX7g4pX-BvNyA,False
3,13944051314,2021-08-30 12:50:00,2021-08-30 12:50:00,BMf1zFkJ1xvPMlOxC7RWTw,Merchant D,8AuFEjK8m-0lxGAOJzOPmg,26Lw20NQtPKb5cyQZBuh1w,False
5,25784401111,2021-06-11 13:21:00,2021-06-11 13:21:00,cjyrXCInVMph0CUUu9JSRg,Merchant B,mtM86EuiwmunVrELKjGT1g,TqcwEBJG9hTBvF_Jxud8qg,False
6,27085470319,2021-09-13 07:51:00,2021-09-13 07:51:00,1yYF_mSnG0ZfjnghlBqEKw,Merchant D,Wqw2dKIsVMc-EMb5WKKruQ,Y1tdy_yS55lYK9zJu-RPMA,False
9,56451872040,2021-07-06 11:28:00,2021-07-06 11:28:00,a0X0Gzw7u7pGwbM2U_7IsA,Merchant A,sh5tqIUcYLC9dZ6hMq-JJg,kJ7AJ9K3mbYwFWaft7JsRA,False


In [119]:
df_graph_features_training_test = df_graph_features.loc[df_graph_features["Global_Date"].dt.month >= 10].copy()

## Generate Graph Features

In [120]:
graph_features = ApateGraphFeatures(df_graph_features_learning)

In [121]:
graph_features.add_graph_features(df_graph_features_training_test)

2023-01-31 14:39:10.919380 : Finished computing IP features...


2023-01-31 14:40:06.225280 : Finished computing Email features...


2023-01-31 14:41:06.622693 : Finished computing Card Holder features...


2023-01-31 14:41:26.712657 : Finished computing Merchant features...


2023-01-31 15:10:49.486504 : Finished computing Transaction features...


In [122]:
df_graph_features_training_test.head()

,psp_reference,Global_Date,Local_Date,card_number,merchant,ip_address,email_address,Target,IPScore,IPScore_ST,...,CHScore_MT,CHScore_LT,MerScore,MerScore_ST,MerScore_MT,MerScore_LT,TrxScore,TrxScore_ST,TrxScore_MT,TrxScore_LT
1,41293916020,2021-12-11 15:20:00,2021-12-11 15:20:00,zgRoL8pxX5LJI-SEPUTMvg,Merchant C,HZkxQvtpScNLgi0fsoSkjA,klwgzdfrnz_6eGsHSi2prQ,False,0.000003,0.000003,...,0.000003,0.000003,0.011340,0.007196,0.011341,0.011340,0.000010,0.000008,0.000010,0.000010
2,31382555255,2021-10-20 01:56:00,2021-10-20 01:56:00,T3BF1uOOqgcwaVRb_ujPQg,Merchant E,HGW5EwBnHVgOrD5xYplBdw,XkMWeLLmXALPcV3ndwzKJw,False,0.000003,0.000003,...,0.000003,0.000003,0.011409,0.010339,0.011376,0.011409,0.000010,0.000010,0.000010,0.000010
4,62343575411,2021-11-14 19:26:00,2021-11-14 19:26:00,jfxS2TIe0gR9vpOUdMKrBw,Merchant E,NaN,Y7tv2i85iUadkEIAPIufgg,False,0.000003,0.000003,...,0.000003,0.000003,0.011409,0.010339,0.011376,0.011409,0.000010,0.000010,0.000010,0.000010
7,42353694618,2021-11-19 17:42:00,2021-11-19 17:42:00,beNTt0aybtIdzY5vp1D99A,Merchant A,NaN,MpwCXjoBbuJoLA3pRc2Zsw,True,0.000003,0.000003,...,0.000029,0.000025,0.045386,0.049368,0.045339,0.045386,0.000045,0.001321,0.000050,0.000045
10,89414101774,2021-12-22 15:28:00,2021-12-22 15:28:00,v5l_VKkQ6uUEp-wexCjLeg,Merchant B,p53LAAaNhLocbYm781iZfA,2_-sJZ7KqnpeTgqadRekTA,False,0.000003,0.000003,...,0.000003,0.000003,0.023279,0.024296,0.023281,0.023279,0.000011,0.000011,0.000011,0.000011


In [123]:
df_graph_features_training_test.to_csv("graph_features_july_december.csv", index=False)

In [124]:
df_graph_features_training_test.columns

Index(['psp_reference', 'Global_Date', 'Local_Date', 'card_number', 'merchant',
       'ip_address', 'email_address', 'Target', 'IPScore', 'IPScore_ST',
       'IPScore_MT', 'IPScore_LT', 'EmailScore', 'EmailScore_ST',
       'EmailScore_MT', 'EmailScore_LT', 'CHScore', 'CHScore_ST', 'CHScore_MT',
       'CHScore_LT', 'MerScore', 'MerScore_ST', 'MerScore_MT', 'MerScore_LT',
       'TrxScore', 'TrxScore_ST', 'TrxScore_MT', 'TrxScore_LT'],
      dtype='object')